In [47]:
#  MODELS

import pandas as pd
import numpy as np

def getwoe(value, label, attr):
    woe = None
    cond1 = attr.ATTR == label
    if 'APPLICANTINCOME' in label or 'COAPPLICANTINCOME' in label:
        the_interval = getinterval(value, label, attr)
        cond2 = attr.VAL == the_interval
    else:
        cond2 = attr.VAL == value
    woe = float(attr.loc[ cond1 & cond2 , 'WOE'])
    return woe

def getinterval(value, label, attr):
    the_interval = None
    if label in ['APPLICANTINCOME_BIN', 'COAPPLICANTINCOME_BIN']:
        intervals = list(attr.loc[attr.ATTR==label, 'VAL'])
        for interval in intervals:
            if value in interval:
                return interval
    return the_interval

def predict(x):
    y = 0.697 + (0.21109)*x[1]+ (0.11063)*x[2]+ (0.59759)*x[3]+ (0.16011)*x[4]+ \
    (0.00595)*x[5]+ (0.15925)*x[6]+ (0.12986)*x[7]+ (0.14358)*x[8]+ (0.03998)*x[9]+ \
    (0.15228)*x[10]+ (0.14257)*x[11]
    return y

attr = pd.read_pickle('Loan_attr.pickle')
test = pd.read_csv('test.csv')

test = test.rename(columns={'ApplicantIncome': 'APPLICANTINCOME_BIN','CoapplicantIncome': 'COAPPLICANTINCOME_BIN',\
                            'LoanAmount' : 'LOANAMOUNT_BIN' , 'Loan_Amount_Term' : 'LOAN_AMOUNT_TERM_BIN'})



# test['Gender'].astype('float64')


test_woe = pd.DataFrame()
test_woe['Loan_ID'] = test['Loan_ID']
fields = ['Gender','Married','Dependents','Education','Self_Employed','Credit_History','Property_Area','APPLICANTINCOME_BIN','COAPPLICANTINCOME_BIN','LOANAMOUNT_BIN','LOAN_AMOUNT_TERM_BIN']
for f in fields:
    test_woe[f'{f}_WOE'] = test[f].apply(getwoe, args=(f, attr))
    
# predict
x = [None]
for f in fields:
    x.append(test_woe[f'{f}_WOE'])
test_woe['y_predict'] = predict(x)
test_woe['prob_status'] = test_woe['y_predict'].map(lambda s: 1 / (1+np.exp(-test_woe['y_predict'])))
test_woe['Loan_Status'] = test_woe['prob_status'].map(lambda s: 1 if s > 0.5 else 0)

TypeError: cannot convert the series to <class 'float'>

In [ ]:
test.info()